# Summary

This notebook allows to reproduce the method results on the KIRP dataset.  
We have analyzed the dataset with both GMM and HDBSCAN algorithms.

In [1]:
import sys
sys.path.append("..")

#GPU configuration
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default

import random
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scripts.data_generator as data_generator
import scripts.feature_ranking as feature_ranking
import scripts.features_2d as features_2d
import scripts.ga as ga
import scripts.preprocess as preprocess
import scripts.ga_evaluation as ga_evaluation
import scripts.bio_analysis as bio_analysis
import tensorflow as tf
from IPython import get_ipython
from tqdm import tqdm
from collections import Counter

plt.ion()
plt.show()

random_state=2
random.seed( random_state )
np.random.seed(random_state)

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



/usr/local/lib/python3.6/dist-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


# Preprocessing

In [ ]:
# truth_column = "tumor_type"
# truth_values = ['type 1', 'type 2']
# filename = "KIRP"

# df = pd.read_csv("../data/rna_data/KIRP.txt", sep = "\t", low_memory=False)
# meta = pd.read_csv("../data/rna_data/KIRP_All_CDEs.txt", sep = "\t", low_memory=False)

# preprocess.preprocess_rna(df,
#                    meta,
#                    truth_column,
#                    truth_values,
#                    filename,
#                    metric='correlation',#'euclidean',
#                    normalize=True)

# Load preprocessed data

## Start here if preprocessing files have been generated

In [ ]:
# filename = "KIRP"

# data = pd.read_pickle(f"../data/rna_data/{filename}.pkl")
# # z_file =f"../data/rna_data/{filename}_Z_correlation.npy"
# # additional_df = pd.read_pickle(f"../data/rna_data/{filename}_additional.pkl")

In [ ]:
data = pd.read_csv("../../deep_clustering/moduledetection-evaluation/data/ecoli_colombos/E.tsv", sep = "\t", index_col= 0)

In [ ]:
data = data.values

In [3]:
# truth = data["y"].values
# data = data.drop("y", axis = 1).values
# n_clusters = len(np.unique(truth))
# Counter(truth), data.shape
truth = None
n_clusters = None

# Subspace clustering

In [ ]:
meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=90,
                                              z_file=None,
                                              metric='correlation',
                                              redundant_threshold=0.6)


In [ ]:
# model_file = "../models/gmm_arl.h5"
# gmm_arl_population, n = features_2d.run(data,
#                                 n_clusters,
#                                 meta_features,
#                                 model_file=model_file,
#                                 theta=0.1,
#                                 add_close_population=False,
#                                 exploration_factor = 5)
# print(gmm_arl_population.shape, n)

In [ ]:
globalResults = {} # Save results for both runs

In [ ]:
method = "adapted_ratkowsky_lance"
score_tolerance=0.009
clustering = "gmm"

round_size = 3
debug = False
ignore_redundant= True
epochs = 10*round_size

sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
}
params = ga.ga_parameters(
    n_clusters,
    data.shape[1],
    truth,
    meta_features,
    method=method,
    truth_methods=['ari'],
    archive_2d=None,
    debug=debug,
    epochs=epochs,
    round_size=round_size,
    sampling = sampling,
    ignore_redundant = ignore_redundant,
    allow_subspace_overlap = False,
    improvement_per_mutation_report = True,
    score_tolerance=score_tolerance,
    clustering = clustering,
    total_maximisation_exploration = 400

)
print(params["sampling_actions"], params["maximisation_sizes"] , params["sampling_prob"])
params

solutions, archive= ga.run(data, params)
solutions.to_pickle(f"../data/{filename}_{clustering}_{method}.pkl")
display(solutions)
# globalResults[f"{clustering}_{method}"] = solutions

In [ ]:
method = "adapted_silhouette"
threshold=0.09
score_tolerance=0.009
clustering = "gmm"

round_size = 1#3
debug = False
ignore_redundant= True
epochs = 1#10*round_size

sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
}
params = ga.ga_parameters(
    n_clusters,
    data.shape[1],
    truth,
    meta_features,
    method=method,
    truth_methods=['ari'],
    archive_2d=None,
    debug=debug,
    epochs=epochs,
    round_size=round_size,
    sampling = sampling,
    ignore_redundant = ignore_redundant,
    allow_subspace_overlap = False,
    improvement_per_mutation_report = True,
    score_tolerance=score_tolerance,
    clustering = clustering,
    total_maximisation_exploration = 400

)
print(params["sampling_actions"], params["maximisation_sizes"] , params["sampling_prob"])
params

solutions, archive= ga.run(data, params)
solutions.to_pickle(f"../data/{filename}_{clustering}_{method}.pkl")
display(solutions)
# globalResults[f"{clustering}_{method}"] = solutions

In [ ]:
method = "adapted_silhouette"
threshold=0.09
score_tolerance=0.009
clustering = "hdbscan"

round_size = 3
debug = False
ignore_redundant= True
epochs = 10*round_size

sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
}
params = ga.ga_parameters(
    n_clusters,
    data.shape[1],
    truth,
    meta_features,
    method=method,
    truth_methods=['ari'],
    archive_2d=None,
    debug=debug,
    epochs=epochs,
    round_size=round_size,
    sampling = sampling,
    ignore_redundant = ignore_redundant,
    allow_subspace_overlap = False,
    improvement_per_mutation_report = True,
    score_tolerance=score_tolerance,
    clustering = clustering,
    total_maximisation_exploration = 500

)
print(params["sampling_actions"], params["maximisation_sizes"] , params["sampling_prob"])
params

solutions, archive= ga.run(data, params)
solutions.to_pickle(f"../data/{filename}_{clustering}_{method}.pkl")
display(solutions)
globalResults[f"{clustering}_{method}"] = solutions

In [ ]:
!ls ../../deep_clustering/moduledetection-evaluation/data/ecoli_colombos/knownmodules/ecoli_regulondb

In [ ]:
for i in range(10):
#     print(f"Len {len(an[i])}")
    pca = PCA(2)
    f = solutions["features"].values[i]
    pca_data = pca.fit_transform(data[:, f])
    input_data = pca_data

#     pred = hdbscan.HDBSCAN(min_cluster_size =10).fit(input_data).labels_
    pred = solutions["partition"].values[i]
    sil = solutions["silhouette"].values[i]

    plt.figure()
    plt.title(f"Clusters : {np.unique(pred).shape[0]}, Len {len(f)}, silhouette {sil}")
    plt.scatter(input_data[:, 0], input_data[:, 1], c = pred)

In [ ]:
import json

with open(
        '../../deep_clustering/moduledetection-evaluation/data/ecoli_colombos/knownmodules/ecoli_regulondb/minimal.json'
) as f:
    an = json.load(f)

In [ ]:
an = [[int(s[1:]) for s in g] for g in an]

In [ ]:
from sklearn import mixture
from sklearn.metrics import silhouette_score
import hdbscan

In [ ]:
for i in range(10):
#     print(f"Len {len(an[i])}")
    pca = PCA(2)
    input_data = data[:, an[i]]
    pca_data = pca.fit_transform(input_data)
    

    pred = hdbscan.HDBSCAN(min_cluster_size =10).fit(input_data).labels_
#     pred = hdbscan.HDBSCAN().fit(input_data).labels_
    sil = silhouette_score(input_data, pred)

    plt.figure()
    plt.title(f"Clusters : {np.unique(pred).shape[0]}, Len {len(an[i])}, sil = {sil}")
    plt.scatter(pca_data[:, 0], pca_data[:, 1], c = pred)
    plt.show()

In [ ]:
method = "adapted_ratkowsky_lance"
threshold=0.09
score_tolerance=0.009
clustering = "hdbscan"

round_size = 3
debug = False
ignore_redundant= True
epochs = 10*round_size

sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
}
params = ga.ga_parameters(
    n_clusters,
    data.shape[1],
    truth,
    meta_features,
    method=method,
    truth_methods=['ari'],
    archive_2d=None,
    debug=debug,
    epochs=epochs,
    round_size=round_size,
    sampling = sampling,
    ignore_redundant = ignore_redundant,
    allow_subspace_overlap = False,
    improvement_per_mutation_report = True,
    score_tolerance=score_tolerance,
    clustering = clustering,
    total_maximisation_exploration = 500

)
print(params["sampling_actions"], params["maximisation_sizes"] , params["sampling_prob"])
params

solutions, archive= ga.run(data, params)
solutions.to_pickle(f"../data/{filename}_{clustering}_{method}.pkl")
display(solutions)
globalResults[f"{clustering}_{method}"] = solutions

# Interpret results

In [ ]:
additional_results, best_subspace_match, best_meta_subspace = bio_analysis.clinical_data_analysis(
    additional_df, solutions, n_clusters)

best_subspace_match

In [ ]:
method = "adapted_silhouette"
threshold=0.1
score_tolerance=0.01
clustering = "hdbscan"

round_size = 3
debug = False
ignore_redundant= True
epochs = 10*round_size

sampling = {
    "ARCHIVE2D": { 
        "ga": 0.3,
        "max": 0.3 },
    "CLOSE": { 
        "ga": 0.4,
        "max": 0.4 },
    "IMP1D": { 
        "ga": 0.2,
        "max": 0.2 },
    "RANDOM": { 
        "ga": 0.1,
        "max": 0.1},
}
params = ga.ga_parameters(
    n_clusters,
    data.shape[1],
    truth,
    meta_features,
    method=method,
    truth_methods=['ari'],
    archive_2d=gmm_arl_population[gmm_arl_population["pred"] > threshold].iloc[:7000],
    debug=debug,
    epochs=epochs,
    round_size=round_size,
    sampling = sampling,
    ignore_redundant = ignore_redundant,
    allow_subspace_overlap = False,
    improvement_per_mutation_report = True,
    score_tolerance=score_tolerance,
    clustering = clustering,
    total_maximisation_exploration = 500

)
print(params["sampling_actions"], params["maximisation_sizes"] , params["sampling_prob"])
params

solutions, archive= ga.run(data, params)
solutions.to_pickle(f"../data/{filename}_{clustering}_{method}.pkl")
display(solutions)
globalResults[f"{clustering}_{method}"] = solutions

# Interpret results

In [ ]:
additional_results, best_subspace_match, best_meta_subspace = bio_analysis.clinical_data_analysis(
    additional_df, solutions, n_clusters)

best_subspace_match

# Supervised analysis

In [ ]:
from sklearn import mixture
import hdbscan

In [ ]:
ranked_features = feature_ranking.supervised_feature_ranking(data, truth, 
                        nbTopFeatures = data.shape[1])
data = data[:, ranked_features]
imp_f = np.arange(50)

In [ ]:
gmm_scores = []
hdbscan_scores = []
for i in range(2, 50):
    input_data = data[:, :i]
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    gmm_scores.append(ari)

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    hdbscan_scores.append(ari)
print(f" GMM ari = {max(gmm_scores)}, {np.argmax(gmm_scores)}")
print(f" HDBSCAN ari = {max(hdbscan_scores)}, {np.argmax(hdbscan_scores)}")

In [ ]:
from sklearn.feature_selection import chi2,  mutual_info_classif, SelectKBest
sel = SelectKBest(mutual_info_classif, k=50).fit_transform(data, truth)
gmm_scores = []
hdbscan_scores = []
for i in range(2, 50):
    input_data = sel[:, :i]
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    gmm_scores.append(ari)

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    hdbscan_scores.append(ari)
print(f" GMM ari = {max(gmm_scores)}, {np.argmax(gmm_scores)}")
print(f" HDBSCAN ari = {max(hdbscan_scores)}, {np.argmax(hdbscan_scores)}")

In [ ]:
input_data = data
gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
pred = gmm.fit_predict(input_data)
ari = adjusted_rand_score(truth, pred)
print(f"GMM ari = {ari}")


pred = KMeans(n_clusters= n_clusters).fit(input_data).labels_
ari = adjusted_rand_score(truth, pred)
print(f"Kmeans ari = {ari}")

In [ ]:
input_data = data
pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
ari = adjusted_rand_score(truth, pred)
print(f"HDBSCAN ari {ari}")

In [ ]:
# Predict on PCA
pca = PCA(2)
pca_data = pca.fit_transform(data)
input_data = pca_data
gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
pred = gmm.fit_predict(input_data)
ari = adjusted_rand_score(truth, pred)
print(f"GMM ari = {ari}")

pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
ari = adjusted_rand_score(truth, pred)
print(f"HDBSCAN ari = {ari}")

# Other methods

In [ ]:
from sklearn import mixture
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
import hdbscan

row = {}
clustering = AffinityPropagation(random_state=5).fit(data)
ari = adjusted_rand_score(truth, clustering.labels_)
print(f"Affinity {ari}")
row["AffinityPropagation"] = ari

clustering = SpectralClustering(n_clusters=n_clusters, random_state=0).fit(data)
ari = adjusted_rand_score(truth, clustering.labels_)
print(f"Spectral {ari}")
row["Spectral"] = ari

clustering = KMeans(n_clusters=n_clusters,random_state=5).fit(data)
ari = adjusted_rand_score(truth, clustering.labels_)
print(f"KMeans {ari}")
row["KMeans"] = ari

gmm = mixture.GaussianMixture(n_components=n_clusters,
              covariance_type="full", random_state=0)
pred = gmm.fit_predict(data[:, :8000])
ari = adjusted_rand_score(truth, pred)
print(f"GMM {ari}")
row["GMM"] = ari

pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(data).labels_
ari = adjusted_rand_score(truth, pred)
print(f"HDBSCAN {ari}")
row["HDBSCAN"] = ari

pca = PCA(2)
pca_data = pca.fit_transform(data)

clustering = KMeans(n_clusters=n_clusters,random_state=5).fit(pca_data)
ari = adjusted_rand_score(truth, clustering.labels_)
print(f"PCA KMeans {ari}")
row["PCA_KMeans"] = ari

gmm = mixture.GaussianMixture(n_components=n_clusters,
              covariance_type="full", random_state=0)
pred = gmm.fit_predict(pca_data)
ari = adjusted_rand_score(truth, pred)
print(f"PCA GMM {ari}")
row["PCA_GMM"] = ari

pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(pca_data).labels_
ari = adjusted_rand_score(truth, pred)
print(f"PCAHDBSCAN {ari}")
row["PCA_HDBSCAN"] = ari
